# Extracting Masks of RIGA Dataset images for Segmentation Trainings
Some information is hidden (changed to [...]) for privacy reasons.

In [ ]:
from PIL import Image, ImageChops
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.morphology import remove_small_objects
from skimage.segmentation import flood, flood_fill
from math import sqrt
import seaborn as sns

In [ ]:
!sudo apt install unzip

In [ ]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [ ]:
# !mkdir RIGA
# !gsutil -m cp -r "gs://[...]/cup-to-disc/datasets/RIGA/RIGA-dataset" RIGA

In [ ]:
!gsutil -m cp -r "gs://[...]/cup-to-disc/datasets_with_masks/RIGA_with_mask" "./"

####Process Functions

In [ ]:
"""
Steps:
0. Copy original numpy matrix to [circles]  
1. Find outer circle's one pixel (must be at top)
    1.1. Find first black pixel inside of outer circle (in [circles])
    1.2. Flood Fill the area between outer and inner circles (fill [disc_minus_cup])
2. Delete outer circle (in [circles])
3. Find inner circle's one pixel (must be at top)
    3.1 Do the same thing in 1.x steps [inside_cup]
4. Sum [inside_cup] and [circles] (dont forget [circles] matrix doesn't have disc line anymore)
    4.1 [cup] = [inside_cup] + [circles]
5. Sum [inside_cup], [disc_minus_cup] and [annots]
    5.1 [disc] = [inside_cup] + [disc_minus_cup] + [annots]
"""


#Find disc
def find_circle_pixel(img_arr):
    for y in range(img_arr.shape[0]):
        for x in range(img_arr.shape[1]):
            if (img_arr[y][x]): return y, x

def find_circle_pixel_inside(img_arr, circle_pixel):
    y, x = circle_pixel
    yd = y; xd = x
    while (img_arr[yd][xd] != 0): yd+=1
    return yd, xd

def fill_circle(arr, start):
    y, x = start
    position = [y, x]
    
    to_fill = []

    going_right = True
    direction_changed = False
    done = False
    while not done:
        y, x = position
        arr[y-quarter_frame:y+quarter_frame,x-quarter_frame:x+quarter_frame] = True
        try:
            test_arr[position[0],position[1]] = 255
        except:
            pass
        if (going_right):
            if not np.all(arr[y-half_frame:y+half_frame,x:x+side_frame]):       #right
                position = (y, x+quarter_frame)
                direction_changed = False
            elif not np.all(arr[y:y+corner_frame, x:x+corner_frame]):           #botom_right
                position = (y+quarter_frame, x+quarter_frame)
                direction_changed = False
                "going_right = False ?"
            elif not np.all(arr[y:y+side_frame, x-half_frame:x+half_frame]):    #bottom
                position = (y+quarter_frame, x)
                direction_changed = False
                "going_right = False ?"
            else: going_right = False; 

            if (direction_changed and not going_right): done = True
            if (not going_right): direction_changed = True
            
            
        else:
            if not np.all(arr[y-half_frame:y+half_frame,x-side_frame:x]):       #left
                position = (y, x-quarter_frame)
                direction_changed = False
            elif not np.all(arr[y:y+corner_frame,x-corner_frame:x]):            #bottom_left
                position = (y+quarter_frame, x-quarter_frame)
                direction_changed = False
                "going_right = True ?"
            elif not np.all(arr[y:y+side_frame, x-half_frame:x+half_frame]):    #bottom
                position = (y+quarter_frame, x)
                direction_changed = False
                "going_right = True ?"
            else: going_right = True

            if (direction_changed and going_right): done = True
            if (going_right): direction_changed = True

        

def erase_circle(arr, start, frame_size = 16):
    y, x = start
    position = [y, x]
    half_frame = frame_size//2+2
    quarter_frame = half_frame//2
    side_frame = frame_size
    corner_frame = half_frame+quarter_frame
    while True:
        next = None
        y, x = position
        arr[y-half_frame-1:y+half_frame+1,x-half_frame-1:x+half_frame+1] = 0

       
        if np.any(arr[y-corner_frame:y,x-corner_frame:x]):                      #top_left
            position = (y-quarter_frame, x-quarter_frame)
        elif np.any(arr[y:y+corner_frame, x:x+corner_frame]):                   #botom_right
            position = (y+quarter_frame, x+quarter_frame)
        elif np.any(arr[y-corner_frame:y,x:x+corner_frame]):                    #top_right
            position = (y-quarter_frame, x+quarter_frame)
        elif np.any(arr[y:y+corner_frame,x-corner_frame:x]):                    #bottom_left
            position = (y+quarter_frame, x-quarter_frame)
        elif np.any(arr[y-half_frame:y+half_frame,x-side_frame:x]):             #left
            position = (y, x-quarter_frame)
        elif np.any(arr[y-half_frame:y+half_frame,x:x+side_frame]):             #right
            position = (y, x+quarter_frame)
        elif np.any(arr[y-side_frame:y,x-half_frame:x+half_frame]):             #top
            position = (y-quarter_frame, x)
        elif np.any(arr[y:y+side_frame, x-half_frame:x+half_frame]):            #bottom
            position = (y+quarter_frame, x)
        else: break
        


In [ ]:
files_with_errors = []

In [ ]:
!mkdir RIGA_with_mask

###1. RIGA 

---



#### BinRushed

In [ ]:
%%capture
!unzip RIGA/RIGA-dataset/BinRushedcorrected.zip -d BinRushed

In [ ]:
def get_prime_and_mask_alternative(im_prime, im_annoted, threshold=100):
    diff_np = np.array(ImageChops.difference(im_prime, im_annoted))
    diff_np[diff_np > 250] = 0
    diff_np = np.sum(diff_np, axis=2)
    annots = remove_small_objects(diff_np>threshold).astype('uint8')*255


    outer_circle_pixel_start = find_circle_pixel(annots)

    inner_circle = annots.copy()

    erase_circle(inner_circle, outer_circle_pixel_start)

    inner_circle_pixel_start = find_circle_pixel(inner_circle)

    outer_circle = annots.copy()
    erase_circle(outer_circle, inner_circle_pixel_start)

    outer_circle_pixel_inside_start = find_circle_pixel_inside(outer_circle, outer_circle_pixel_start)
    inner_circle_pixel_inside_start = find_circle_pixel_inside(inner_circle, inner_circle_pixel_start)

    disc = flood_fill(outer_circle, outer_circle_pixel_inside_start, 255)
    cup = flood_fill(inner_circle, inner_circle_pixel_inside_start, 255)

    return disc, cup

def get_prime_and_mask(im_prime, im_annoted, threshold=100):
    diff_np = np.array(ImageChops.difference(im_prime, im_annoted))
    diff_np[diff_np > 250] = 0
    diff_np = np.sum(diff_np, axis=2)
    annots = remove_small_objects(diff_np>threshold).astype('uint8')*255

    outer_circle_pixel_start = find_circle_pixel(annots)
    inner_circle = flood_fill(annots, outer_circle_pixel_start, 0)

    inner_circle_pixel_start = find_circle_pixel(inner_circle)
    outer_circle = flood_fill(annots, inner_circle_pixel_start, 0)

    outer_circle_pixel_inside_start = find_circle_pixel_inside(outer_circle, outer_circle_pixel_start)
    inner_circle_pixel_inside_start = find_circle_pixel_inside(inner_circle, inner_circle_pixel_start)

    disc = flood_fill(outer_circle, outer_circle_pixel_inside_start, 255)
    cup = flood_fill(inner_circle, inner_circle_pixel_inside_start, 255)
    
    return disc, cup

In [ ]:
thresholds = [120, 100, 80, 70, 60, 50, 40, 30, 20, 10, 5]


In [ ]:
binrushed_dir_names = ['BinRushed1-Corrected', 'BinRushed2', 'BinRushed3', 'BinRushed4']
for binrushed_dir in binrushed_dir_names: 
    to_create = f"RIGA_with_mask/{binrushed_dir}"
    !mkdir $to_create

In [ ]:
for binrushed_dir in tqdm(binrushed_dir_names):
    path = f"BinRushed/BinRushed/{binrushed_dir}"
    br_images = [_ for _ in os.listdir(f"BinRushed/BinRushed/{binrushed_dir}")]
    annot_images_filenames = [_ for _ in br_images if _.find("prime") == -1]
    n = max([int(_.split("prime")[0].replace("image", "")) for _ in br_images if _.find("prime") != -1])+1
    for i in tqdm(range(1, n)):
        fp = f"RIGA_with_mask/{binrushed_dir}/image{i}prime.bmp"

        im_prime_filename = f"image{i}prime.jpg"
        im_prime_path = f"{path}/{im_prime_filename}"
        im_prime = None
        
        # im_prime.save(fp, quality=100)
        
        for j in range(1, 7):
            if f"RIGA_with_mask/{binrushed_dir}/image{i}-{j}" not in exceptional_ratios: continue

            im_annot_path = f"image{i}-{j}"
            
            if (im_prime == None): im_prime = Image.open(im_prime_path)

            try:
                im_annot = Image.open(f"{path}/{im_annot_path}.jpg")
            except FileNotFoundError:
                im_annot = Image.open(f"{path}/{im_annot_path}.tif")
            
            try:
                ratio_valid = False
                for threshold in thresholds:
                    disc, cup = get_prime_and_mask(im_prime, im_annot, threshold=threshold)
                    total_pixels = cup.shape[0]*cup.shape[1]
                    cup_pixels = np.sum(cup == 255)
                    disc_pixels = np.sum(disc == 255)

                    ratio = sqrt(cup_pixels/disc_pixels)
                    ratio_valid = ((ratio > 0.1) and (ratio < 0.9))


                    ratio_valid = ratio_valid and (cup_pixels/total_pixels <= 0.8) 
                    ratio_valid = ratio_valid and (disc_pixels/total_pixels <= 0.8)
                    
                    if ratio_valid: break

                if (not ratio_valid):
                    for threshold in thresholds:
                        disc, cup = get_prime_and_mask_alternative(im_prime, im_annot, threshold=threshold)
                        
                        total_pixels = cup.shape[0]*cup.shape[1]
                        cup_pixels = np.sum(cup == 255)
                        disc_pixels = np.sum(disc == 255)

                        ratio = sqrt(cup_pixels/disc_pixels)
                        ratio_valid = ((ratio > 0.1) and (ratio < 0.9))


                        ratio_valid = ratio_valid and (cup_pixels/total_pixels <= 0.8) 
                        ratio_valid = ratio_valid and (disc_pixels/total_pixels <= 0.8)

                        if ratio_valid: break

                if (not ratio_valid):
                    print(f"Error while parsing (ratio={ratio}): {path}/{im_annot_path}")
                    files_with_errors.append(f"Ratio error (ratio={ratio}): {path}/{im_annot_path}")
                    continue #to next annotation
            
            except TypeError:
                print(f"Line collision: {path}/{im_annot_path}")
                files_with_errors.append(f"Line collision: {path}/{im_annot_path}")
                continue #to next annotation

            im_disc = Image.fromarray(disc)
            im_cup = Image.fromarray(cup)

            fp = f"RIGA_with_mask/{binrushed_dir}/image{i}-{j}_disc.bmp"
            im_disc.save(fp, quality=100)

            fp = f"RIGA_with_mask/{binrushed_dir}/image{i}-{j}_cup.bmp"
            im_cup.save(fp, quality=100)

In [ ]:
"""Send All BinRushed to storage"""
for binrushed_dir in binrushed_dir_names:
    sub_dir = f"RIGA_with_mask/{binrushed_dir}"
    !gsutil -m cp -r $sub_dir "gs://[...]/cup-to-disc/datasets_with_masks/RIGA_with_mask"

In [ ]:
!rm -r BinRushed
# !rm RIGA/RIGA-dataset/BinRushed.zip

####Messidor


In [ ]:
%%capture
!unzip RIGA/RIGA-dataset/MESSIDOR.zip -d Messidor

In [ ]:
# binrushed_dir_names = ['MESSIDOR']
# for binrushed_dir in binrushed_dir_names:
#     to_create = f"RIGA_with_mask/{binrushed_dir}"
#     !mkdir $to_create

In [ ]:
messidor_dir = "MESSIDOR"
path = f"Messidor/{messidor_dir}"
br_images = [_ for _ in os.listdir(path)]
annot_images_filenames = [_ for _ in br_images if _.find("prime") == -1]
n = max([int(_.split("prime")[0].replace("image", "")) for _ in br_images if _.find("prime") != -1])+1
print("total images:", n)
for i in tqdm(range(1, n)):
    fp = f"RIGA_with_mask/{messidor_dir}/image{i}prime.bmp"

    im_prime_filename = f"image{i}prime.tif"
    im_prime_path = f"{path}/{im_prime_filename}"
    im_prime = None
    
    # im_prime.save(fp, quality=100)

    # continue
    for j in range(1, 7):
        im_annot_path = f"image{i}-{j}"
        if f"RIGA_with_mask/{messidor_dir}/image{i}-{j}" not in exceptional_ratios: continue
        if im_prime == None: im_prime = Image.open(im_prime_path)

        try:
            im_annot = Image.open(f"{path}/{im_annot_path}.jpg")
        except FileNotFoundError:
            im_annot = Image.open(f"{path}/{im_annot_path}.tif")
        
        try:
            ratio_valid = False
            for threshold in thresholds:
                disc, cup = get_prime_and_mask_alternative(im_prime, im_annot, threshold=threshold)
                total_pixels = cup.shape[0]*cup.shape[1]
                cup_pixels = np.sum(cup == 255)
                disc_pixels = np.sum(disc == 255)

                ratio = sqrt(cup_pixels/disc_pixels)
                ratio_valid = ((ratio > 0.1) and (ratio < 0.9))


                ratio_valid = ratio_valid and (cup_pixels/total_pixels <= 0.8) 
                ratio_valid = ratio_valid and (disc_pixels/total_pixels <= 0.8)
                if ratio_valid: break

            if (not ratio_valid):
                for threshold in thresholds:
                    disc, cup = get_prime_and_mask(im_prime, im_annot, threshold=threshold)
                    total_pixels = cup.shape[0]*cup.shape[1]
                    cup_pixels = np.sum(cup == 255)
                    disc_pixels = np.sum(disc == 255)

                    ratio = sqrt(cup_pixels/disc_pixels)
                    ratio_valid = ((ratio > 0.1) and (ratio < 0.9))

                    ratio_valid = ratio_valid and (cup_pixels/total_pixels <= 0.8) 
                    ratio_valid = ratio_valid and (disc_pixels/total_pixels <= 0.8)

                    if ratio_valid: break

            if (not ratio_valid):
                print(f"Error while parsing (ratio={ratio}): {path}/{im_annot_path}")
                files_with_errors.append(f"Ratio error (ratio={ratio}): {path}/{im_annot_path}")
                continue #to next annotation
        
        except TypeError:
            print(f"Line collision: {path}/{im_annot_path}")
            files_with_errors.append(f"Line collision: {path}/{im_annot_path}")
            continue #to next annotation

        

        im_disc = Image.fromarray(disc)
        im_cup = Image.fromarray(cup)

        fp = f"RIGA_with_mask/{messidor_dir}/image{i}-{j}_disc.bmp"
        im_disc.save(fp, quality=100)

        fp = f"RIGA_with_mask/{messidor_dir}/image{i}-{j}_cup.bmp"
        im_cup.save(fp, quality=100)

In [ ]:
"""Send Messidor to storage"""
!gsutil -m cp -r RIGA_with_mask/MESSIDOR gs://[...]/cup-to-disc/datasets_with_masks/RIGA_with_mask

In [ ]:
!rm -r Messidor
!rm RIGA/RIGA-dataset/MESSIDOR.zip

####Magrabia

In [ ]:
%%capture
!unzip RIGA/RIGA-dataset/Magrabia.zip -d Magrabia

In [ ]:
magrabia_dir_names = ['MagrabiaMale', 'MagrabiaFemale']
for magrabia_dir in magrabia_dir_names:
    to_create = f"RIGA_with_mask/{magrabia_dir}"
    !mkdir $to_create

In [ ]:
for magrabia_dir in  magrabia_dir_names:
    path = f"Magrabia/Magrabia/{magrabia_dir}"
    br_images = [_ for _ in os.listdir(path)]
    annot_images_filenames = [_ for _ in br_images if _.find("prime") == -1]
    n = max([int(_.split("prime")[0].replace("image", "")) for _ in br_images if _.find("prime") != -1])+1
    print(f"total images for {magrabia_dir}:", n)
    for i in tqdm(range(1, n)):
        fp = f"RIGA_with_mask/{magrabia_dir}/image{i}prime.bmp"

        
        im_prime_filename = f"image{i}prime.tif"
        im_prime_path = f"{path}/{im_prime_filename}"
        im_prime = None
        # im_prime.save(fp, quality=100)
        
        for j in range(1, 7):

            if f"RIGA_with_mask/{magrabia_dir}/image{i}-{j}" not in exceptional_ratios: continue
            
            im_annot_path = f"image{i}-{j}"
            
            if (im_prime == None): im_prime = Image.open(im_prime_path)

            try:
                im_annot = Image.open(f"{path}/Image{i}-{j}.tif")
            except FileNotFoundError:
                im_annot = Image.open(f"{path}/image{i}-{j}.tif")
            
            try:
                ratio_valid = False
                for threshold in thresholds:
                    disc, cup = get_prime_and_mask(im_prime, im_annot, threshold=threshold)
                    total_pixels = cup.shape[0]*cup.shape[1]
                    cup_pixels = np.sum(cup == 255)
                    disc_pixels = np.sum(disc == 255)

                    ratio = sqrt(cup_pixels/disc_pixels)
                    ratio_valid = ((ratio > 0.1) and (ratio < 0.9))

                    ratio_valid = ratio_valid and (cup_pixels/total_pixels <= 0.8) 
                    ratio_valid = ratio_valid and (disc_pixels/total_pixels <= 0.8)

                    if ratio_valid: break

                if (not ratio_valid):
                    for threshold in thresholds:
                        disc, cup = get_prime_and_mask_alternative(im_prime, im_annot, threshold=threshold)
                        
                        total_pixels = cup.shape[0]*cup.shape[1]
                        cup_pixels = np.sum(cup == 255)
                        disc_pixels = np.sum(disc == 255)

                        ratio = sqrt(cup_pixels/disc_pixels)
                        ratio_valid = ((ratio > 0.1) and (ratio < 0.9))


                        ratio_valid = ratio_valid and (cup_pixels/total_pixels <= 0.8) 
                        ratio_valid = ratio_valid and (disc_pixels/total_pixels <= 0.8)

                        if ratio_valid: break

                if (not ratio_valid):
                    print(f"Error while parsing (ratio={ratio}): {path}/{im_annot_path}")
                    files_with_errors.append(f"Ratio error (ratio={ratio}): {path}/{im_annot_path}")
                    continue #to next annotation
        
            except TypeError:
                print(f"Line collision: {path}/{im_annot_path}")
                files_with_errors.append(f"Line collision: {path}/{im_annot_path}")
                continue #to next annotation

            im_disc = Image.fromarray(disc)
            im_cup = Image.fromarray(cup)

            fp = f"RIGA_with_mask/{magrabia_dir}/image{i}-{j}_disc.bmp"
            im_disc.save(fp, quality=100)

            fp = f"RIGA_with_mask/{magrabia_dir}/image{i}-{j}_cup.bmp"
            im_cup.save(fp, quality=100)

In [ ]:
"""Send All BinRushed to storage"""
for binrushed_dir in binrushed_dir_names:
    sub_dir = f"BinRushed/BinRushed/{binrushed_dir}"
    !gsutil -m cp -r $sub_dir "gs://[...]/cup-to-disc/datasets_with_masks/RIGA_with_mask"

####Send to Storage

In [ ]:
# "DELETE MASKS"
# !gsutil -m rm -r gs://[...]/cup-to-disc/datasets_with_masks/RIGA_with_mask


In [ ]:
"""Send All Parseds to storage"""
!gsutil -m cp -r RIGA_with_mask gs://[...]/cup-to-disc/datasets_with_masks

In [ ]:
# !rm -r Magrabia

###TEST

In [ ]:
ratios = []
exceptional_ratios = {}

In [ ]:
for group_dir in tqdm(os.listdir("RIGA_with_mask")):
    # if (group_dir.find("BinRushed") == -1): continue
    prime_image_names = [_ for _ in os.listdir(f"RIGA_with_mask/{group_dir}") if _.find("prime") != -1]
    path = f"RIGA_with_mask/{group_dir}"
    for prime_image_name in tqdm(prime_image_names):
        image_n = prime_image_name.split("prime")[0].replace("image", "")
        for i in range(1, 7):
            mask_path = f"{path}/image{image_n}-{i}"
            try:
                im_cup = Image.open(f"{mask_path}_cup.bmp")
                im_disc = Image.open(f"{mask_path}_disc.bmp")
                
                cup = np.array(im_cup)
                disc = np.array(im_disc)

                total_pixels = cup.shape[0]*cup.shape[1]
                cup_pixels = np.sum(cup == 255)
                disc_pixels = np.sum(disc == 255)

            except FileNotFoundError:
                exceptional_ratios[mask_path] = "NotFound"
            ratio = sqrt(cup_pixels/disc_pixels)
            if (ratio < 0.1 or ratio > 0.9):
                exceptional_ratios[mask_path] = ratio
            elif (cup_pixels/total_pixels > 0.8) or (disc_pixels/total_pixels > 0.8):
                exceptional_ratios[mask_path] = "Overflowed"
            ratios.append(ratio)
                

In [ ]:
plt.figure(figsize=(15,10))
sns.distplot(ratios, hist=False, rug=True);

In [ ]:
exceptional_ratios